# Surface code compilation

Semester project: 'A practical introduction into surface code compilation' at
[LSI](https://www.epfl.ch/labs/lsi/), EPFL.

Professor: [Prof. Giovanni De Micheli](https://people.epfl.ch/giovanni.demicheli/?lang=en)\
Supervisor: [Mathias Soeken, Ph.D.](https://people.epfl.ch/mathias.soeken?lang=en)\
Student: [Nicolas Bähler](https://people.epfl.ch/nicolas.bahler?lang=en)

## Table of contents

- [Introduction](#introduction)
- [Surface codes](#surface-codes)
  - [Section 1.1](#section_1_1)
    - [Sub Section 1.1.1](#sub_section_1_1_1)
- [Universality of the gate set](#universality-of-the-gate-set)
- [Compilation of long-range CNOTs](#compilation-of-long-range-cnots)
- [Further optimizations](#further-optimizations)
- [Conclusion](#conclusion)

## [Introduction](#introduction)

If you want to go deeper, check out [this
paper](https://arxiv.org/pdf/2110.11493.pdf) [\[1\]](#references). It’s the
basis of this post and definitely a fun read!

## [Surface codes](#surface-codes)

![alt text](figures/operations.png)\
_Elementary surface code operations:
Single-qubit preparation in the X basis (i), and the Z basis (ii), destructive
single-qubit measurement, in the X basis (iii), and the Z basis (iv), two-qubit
joint measurement of XX (v) and ZZ (vi), a move of a logical qubit from one
patch to an unused patch (vii), two-qubit preparation (viii) and destructive
measurement (ix) in the Bell basis and finally a Hadamard gate using 3 ancilla
logical qubits (x). [\[1\]](#references)_

For anyone familiar with the Quantum Circuit Model, this colorful picture above
seems very different from quantum circuits, how we know and love them! But there
is reason to this ‘madness’. Surface codes, a special kind of Error Correcting
Codes, hold the promise to make fault-tolerant and large-scale quantum computing
possible (in the near future, of course …). The basic idea behind surface codes
is simple. Assume a planar grid of qubits and only neighbor interactions between
qubits are possible. Square patches of _physical_ qubits, are conceptually
combined into one _logical_ qubit in a way that makes this logical qubit error
correctable. The physical qubits inside a patch are split up into data qubits
and ancilla qubits. Through a set of smart measurements of the ancilla qubits
only, one can infer errors that might have occurred on the data qubits, which
can then be corrected. In the picture above, each cell of the gray chessboard
represents one such logical qubit. On top of that fault-tolerant structure, one
can build a set of gates, the elementary ones depicted quite colorfully above.
By composing multiple elementary gates, the gate set can be extended to be
universal.

## [Universality of the gate set](#universality-of-the-gate-set)

We want to implement the preparations and measurements of individual qubits in
the computational basis (Z basis), controlled-not (CNOT), Pauli-X, -Y, and -Z,
Hadamard (H), Phase S and T gates.

The set of elementary surface code operations allows to directly implement all
but the Pauli and phase gates. As Pauli gates commute with the remaining gates,
we can track them classically and adapt the final measurements accordingly.
Hence, no need to implement those physically. In order to also implement the
phase gates, we rely on a technique that was given a most suiting name: _Magic
State Distillation_.

This technique uses highly specialized circuit sections, called _Magic State
Factories_, which can be in a region of the 2D grid of logical qubits far apart
from the main section where the quantum algorithm is implemented. In those
factories, initial low quality quantum states are repeatedly processed to
improve their quality until the resulting state is close enough to a target
state, hence the term ‘distillation’. Consuming such magic states, both phase
gates (S and T) can be fabricated. For S and T gates to be useable in the
quantum circuit, though, we still face one challenge: How do we ‘transport’
those magic states from their factories (potentially in a far away region of the
grid) to the location where we need them to implement either of the phase gates?

This problem is closely related to another aspect of the surface code
architecture. As opposed to conventional quantum circuits, geometrical
constraints are taken into account here. What do we do if a two-qubit operation
has to be performed between two qubits which are physically far apart from each
other? The simplest idea is to use a bunch of SWAP gates (can be easily
implemented using two move operations) to bring the two quantum states, the
operands of the two-qubit operation, physically close together. But there is a
catch, we require $Ω(n)$ SWAP gates, where $n$ is the distance between the
operands, to do so.

Can we do something smarter? The answer is yes, but it comes at the cost of some
compilation complexity! I’m going to illustrate this with the example of
long-range CNOTs, meaning CNOTs between physically distant qubits.

## [Compilation of long-range CNOTs](#compilation-of-long-range-cnots)

First, let’s look at how a local CNOT can be expressed in terms of elementary
operations.

![alt text](figures/local_cnot.png)\
_The arrow (at a for example) indicates that
the result of a measurement is stored classically in a variable, which can then
be used to conditionally perform additional operations if the variable equals
$|1⟩$, indicated by the equal sign (at b for example). The CNOT additionally
consumes a $|+⟩$ state. [\[1\]](#references)_

The key idea, to extend this circuitry to support long-range without using SWAP
gates, is to entangle the operands using Bell preparation.

![alt text](figures/long_range_cnot.png)\
_In this figure, the Pauli corrections are
omitted and just hidden inside the f-expression, which is conditioned on all
the measuring results. [\[1\]](#references)_

Note the constant depth of the circuit, although $|ϕ⟩$ and $|ψ⟩$ can be far
apart! But those entangled chains add geometrical complexity. When going back
to the 2D grid structure of logical qubits, the Bell chains can be seen as paths
from target to control qubits on the grid. Further, we’d like to execute
operations in parallel. Hence, intersections with other Bell chains have to be
avoided, otherwise we would mess up the entanglement. This is the additional
twist in the compilation pass!

To make our lives easier, we are subdividing the 2D grid of logical qubits into
data and ancilla qubits. CNOTs can only be performed on data qubits, where the
ancillas are going to enable the Bell chains between the control and the target
qubit, also referred to as terminals. Further, we can define a graph structure,
called _Operator Graph_, highlight in blue below.

![alt text](figures/operator_paths.png.png)\
_Black squares are data qubits, the other ones
are ancilla qubits. Further, control data qubits (B4 for example) are only
vertically connected to its neighboring ancillas, similarly target data qubits
(D2 for example) are only adjacent to their neighbors vertically. These
limitations are due to the nature of the underling surface code.
[\[1\]](#references)_

Assume now that we want to execute a bunch of CNOTs. In a first step, we want to
connect as many of the terminals with edge-disjoint paths in the operator graph
as possible, as all those CNOTs will be executable in parallel. Note that no
data qubit can be part of the interior of the paths, and each terminal can only
be part of one path. Should a data qubit be used multiple times, there is no way
of running those CNOTs in parallel. They will have to be run in sequential
phases that I call epochs.

![alt text](figures/paths.png)\
_An example of one epoch containing each data
qubit at most once in any path and being connected via edge-disjoint paths over
ancilla qubits only. [\[1\]](#references)_

Now, we still have a problem, remember when I said that those entangled Bell
chains between terminals can’t intersection each other? But that’s exactly what
we have here, look at E5 above for example. In a next step, we need to cut up
and divide the paths into two phases in a way that all path segments in each of
the two phases are vertex-disjoint. This ensures that no Bell chains intersect
each other, at least in the same phase.

![alt text](figures/phases_paths.png)\
_An example of one epoch that has been split up
into two phases, ensuring vertex-disjoint paths. [\[1\]](#references)_

We maximally need two phases per epoch (we might get lucky and have only none
overlapping paths in a given epoch) and we have seen that each CNOT can be run
in constant depth. Hence, each phase has constant depth as well and, thus, we
are limited by the number of epochs. This number is intrinsic to the (re)usage
and of data qubits. There is no dependence on the distance between terminals,
and we can run many CNOTs in parallel. This is a big leap!

The same idea of using such Bell chains can be applied to transporting the
aforementioned magic states from their factories to the location on the grid
where we need to implement phase gates, thus, also at constant depth.

In the paper, the main algorithm combines this procedure described above with a
greedy algorithm based on shortest paths and finally picks the result from the
one that produces fewer epochs.

## [Further optimizations](#further-optimizations)

One can think of further optimizations steps, that are not discussed in the
paper. One idea would be to compute a smart mapping of qubits onto the grid,
which leads to shorter paths and fewer intersections. Another one is instruction
reordering, similar to what modern CPUs do to improve instruction level
parallelism.

## [Conclusion](#conclusion)

The goal of my project is to finally compile down all those operations into Q#
code, which would allow comparing the optimized version with an untouched
version of the circuit via simulation and check correctness and performance
uplift. But this will actually be tricky as the surface code, even ignoring the
fact that each logical qubit is composed of multiple physical qubits, requires
many qubits …

This fact allows me to draw a nice final conclusion. The technique I sketched
out for you is fascinating, but it needs many qubits. As new quantum chips with
more and more qubits are being develop at an every increasing pace, I think the
in a ‘near future’, surface codes will be the basis of fault-tolerant large
scale quantum computing. And hence, surface code compilation is a ‘hot topic’ to
work on right now! I hope that I could spark your interest on the matter!

## [References](#references)

[1] https://arxiv.org/pdf/2110.11493.pdf
